# Laboratorio 2: Limpieza de datos

In [1]:
"""Importar Librerias """

"""crear dataframes"""
import pandas as pd

"""crear vectores, matrices y calculos estadisticos"""
import numpy as np

""" modulo para aplicar metodos a los datos de tipo texto"""
import string

""" crear tecnicas de imputacion de datos con estrategias"""
from sklearn.impute import SimpleImputer

""" imputar valores faltantes utilizando la tecnica de k-vecinos cercanos"""
from sklearn.impute import KNNImputer


## 1. Encontrar y describir problemas del dataset

In [2]:
""" importar el dataset """
df = pd.read_csv('data_act_01.csv',sep=';')

""" visualizar diez primeros registros """
df.head(10)

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallTime,CallDateTime,Disposition,Address,City,State,AgencyId,Range,AddressType
0,160903280,Assault / Battery,2016-03-30T00:00:00,18:42,2016-03-30T18:42:00,REP,100 Block Of Chilton Av,San Francisco,CA,1,NaN,Premise Address
1,160912272,Homeless Complaint,2016-03-31T00:00:00,15:31,2016-03-31T15:31:00,GOA,2300 Block Of Market St,San Francisco,CA,1,NaN,Premise Address
2,160912590,Susp Info,2016-03-31T00:00:00,16:49,2016-03-31T16:49:00,GOA,2300 Block Of Market St,San Francisco,CA,1,NaN,Premise Address
3,160912801,Report,2016-03-31T00:00:00,17:38,2016-03-31T17:38:00,GOA,500 Block Of 7th St,San Francisco,CA,1,NaN,Premise Address
4,160912811,594,2016-03-31T00:00:00,17:42,2016-03-31T17:42:00,REP,Beale St/bryant St,San Francisco,CA,1,NaN,Intersection
5,160913003,Ref'd,2016-03-31T00:00:00,18:29,2016-03-31T18:29:00,GOA,16th St/pond St,San Francisco,CA,1,NaN,Intersection
6,160913050,Homeless Complaint,2016-03-31T00:00:00,18:43,2016-03-31T18:43:00,ADV,Berwick Pl/harrison St,San Francisco,CA,1,NaN,Intersection
7,160913056,Homeless Complaint,2016-03-31T00:00:00,18:47,2016-03-31T18:47:00,HAN,Florida St/mariposa St,San Francisco,CA,1,NaN,Intersection
8,160913078,Agg Assault / Adw Dv,2016-03-31T00:00:00,18:52,2016-03-31T18:52:00,ND,100 Block Of Genebern Wy,San Francisco,CA,1,NaN,Premise Address
9,160913103,Encampment,2016-03-31T00:00:00,18:57,2016-03-31T18:57:00,ADV,2700 Block Of Folsom St,San Francisco,CA,1,NaN,Premise Address


In [3]:
""" obtener informacion de variables del dataset """
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CrimeId                10051 non-null  int64  
 1   OriginalCrimeTypeName  10051 non-null  object 
 2   OffenseDate            10051 non-null  object 
 3   CallTime               10051 non-null  object 
 4   CallDateTime           10051 non-null  object 
 5   Disposition            10051 non-null  object 
 6   Address                10051 non-null  object 
 7   City                   9730 non-null   object 
 8   State                  10048 non-null  object 
 9   AgencyId               10051 non-null  object 
 10  Range                  0 non-null      float64
 11  AddressType            10051 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 942.4+ KB


In [4]:
""" comprobar existencia de registros duplicados """
df.duplicated().sum()

0

**Interpretación del dataset**: <br>
El dataset cuenta con 12 caracteristicas y 10051 muestras recopiladas. A simple vista la mayoría de las columnas poseen el tipo de dato objeto. Sin embargo, la característica 'CrimeId' y 'Range' son de tipo de dato numerico. Algunas columnas poseen datos faltantes, especialmente la caracteristica 'Range' que no contiene ni un valor en sus filas. Por otro lado, se utiliza el método *duplicated()* para verificar si existen registros duplicados en el dataset.

**Definir variables**

In [5]:
""" listar caracteristicas del dataset"""
caracteristicas = ['CrimeId','OriginalCrimeTypeName','OffenseDate','CallTime','CallDateTime','Disposition','Address','City','State','AgencyId','Range','AddressType']

### 1.1 Encontrar datos faltantes

In [6]:
""" comprobar cantidad de datos faltantes en el dataset """
df.isnull().sum()

CrimeId                      0
OriginalCrimeTypeName        0
OffenseDate                  0
CallTime                     0
CallDateTime                 0
Disposition                  0
Address                      0
City                       321
State                        3
AgencyId                     0
Range                    10051
AddressType                  0
dtype: int64

El resultado arrojado muestra las columnas que poseen valores NaN. Existen 3 características que poseen esos datos desconocidos, las cuales son City, State y Range. El resto de columnas se encuentra con datos completos. Se debe corregir esos valores desconocidos y en el caso de la característica Range tiene una particularidad la cual es que todos sus filas son datos faltantes. En este ultimo caso, la opción es eliminar la columna ya que no contiene ningún dato que permita hacer una referencia para luego reemplazar.

In [7]:
""" encontrar el promedio de los valores faltantes """
np.round(df.isnull().mean(),4)

CrimeId                  0.0000
OriginalCrimeTypeName    0.0000
OffenseDate              0.0000
CallTime                 0.0000
CallDateTime             0.0000
Disposition              0.0000
Address                  0.0000
City                     0.0319
State                    0.0003
AgencyId                 0.0000
Range                    1.0000
AddressType              0.0000
dtype: float64

### 1.2 Encontrar valores unicos

In [8]:
""" encontrar valores unicos de las caracteristicas """
def valores_unicos_etiquetas(n_caracteristicas):
    """ recorrer lista de características """
    for i in n_caracteristicas:
        """mostrar caracteristica y sus valores """
        display(i, df[i].unique(), '----')

In [9]:
""" ejecutar funcion para mostrar valores unicos"""
valores_unicos_etiquetas(caracteristicas)

'CrimeId'

array([160903280, 160912272, 160912590, ..., 160964227, 160964229,
       160964249], dtype=int64)

'----'

'OriginalCrimeTypeName'

array(['Assault / Battery', 'Homeless Complaint', 'Susp Info', 'Report',
       '594', "Ref'd", 'Agg Assault / Adw Dv', 'Encampment', 'Burglary',
       'Suspicious Person', 'Ip', 'Arrest Made', 'Tent',
       'Intoxicated Person', 'Guitar', 'H&r Veh Accident', 'Music',
       'Person Dumping', 'Vandalism', 'Petty Theft', '415',
       'Noise Nuisance', 'Fraud', 'Rep', 'Meet W/citizen',
       'Prisoner Transport', 'Poss', 'Trespasser', 'Stolen Vehicle',
       'Fight No Weapon', 'Check For Ped', 'Juvenile Disturbance',
       'Party', 'Lltd', 'Robbery', '601', '500e', '913', 'Traffic Stop',
       '909', 'Person W/gun', 'Suspicious Vehicle', 'Fight No Weapon Dv',
       'Audible Alarm', 'Parking', 'Pay', 'Silent Holdup Alarm',
       'Passing Call', 'Sleeper', 'Family', 'Thumping', 'Loud Music',
       '916', 'Att', 'Threats / Harassment', 'Boombox',
       'Traf Violation Tow', 'Dw', 'Bar Check', 'Suicide Attempt', 'Susp',
       'Crosswalk', 'Casing/917', 'Wireless-Drop', 'Uc Hastin

...

array([nan])

'----'

'AddressType'

array(['Premise Address', 'Intersection', 'Common Location',
       'Geo-Override', 'Intersectioon', '1'], dtype=object)

'----'

In [9]:
""" encontrar el numero de veces que se repiten los valores unicos de cada una de las caracteristicas """
def valores_unicos_etiquetas(n_caracteristicas):
    for i in n_caracteristicas:
        display(df.groupby(i).size().reset_index(name='Cantidad'))

In [10]:
""" ejecutar funcion para encontrar el nro de veces que se repite un valor en las filas de las caracteristicas"""
valores_unicos_etiquetas(caracteristicas)

,CrimeId,Cantidad
0,160903280,1
1,160912272,1
2,160912590,1
3,160912801,1
4,160912811,1
...,...,...
10042,160964210,1
10043,160964216,1
10044,160964227,1
10045,160964229,1


,OriginalCrimeTypeName,Cantidad
0,***901 Req***,1
1,**Tazer**,1
2,*909*,1
3,.25/500e,2
4,.52,1
...,...,...
570,Wz,12
571,X,17
572,X 594,1
573,Yelling,3


,OffenseDate,Cantidad
0,2013-04-04T00:00:00,1
1,2016-03-30T00:00:00,1
2,2016-03-31T00:00:00,87
3,2016-04-01T00:00:00,2224
4,2016-04-02T00:00:00,2259
5,2016-04-03T00:00:00,2193
6,2016-04-04T00:00:00,2255
7,2016-04-05T00:00:00,1030
8,2025-04-04T00:00:00,1


,CallTime,Cantidad
0,0:00,10
1,0:01,5
2,0:02,6
3,0:03,11
4,0:04,11
...,...,...
1411,9:55,7
1412,9:56,5
1413,9:57,7
1414,9:58,11


,CallDateTime,Cantidad
0,2016-03-30T18:42:00,1
1,2016-03-31T15:31:00,1
2,2016-03-31T16:49:00,1
3,2016-03-31T17:38:00,1
4,2016-03-31T17:42:00,1
...,...,...
5111,2016-04-05T23:32:00,2
5112,2016-04-05T23:38:00,1
5113,2016-04-05T23:43:00,1
5114,2016-04-05T23:48:00,2


...

,City,Cantidad
0,S,1
1,Brisbane,1
2,Daly City,5
3,Presidio,3
4,SAN FRANCISCO,1
5,San Francisco,9665
6,Treasure Isla,51
7,Yerba Buena,3


,State,Cantidad
0,CA,10048


,AgencyId,Cantidad
0,1,10048
1,CA,3


,Range,Cantidad


,AddressType,Cantidad
0,1,3
1,Common Location,818
2,Geo-Override,469
3,Intersection,3701
4,Intersectioon,1
5,Premise Address,5059


### 1.3 Eliminacion de datos desconocidos y datos faltantes (Nan)

In [11]:
""" eliminar columna Range del conjunto de datos porque contiene Nan en todas sus filas"""
df.drop(columns='Range', axis=1, inplace=True)

In [12]:
""" agrupar muestras que poseen datos desconocidos en las columnas que tienen un mismo valor en su mayoria de muestras """
df.groupby(['AgencyId','AddressType']).count()

CrimeId  OriginalCrimeTypeName  OffenseDate  \
AgencyId AddressType                                                    
1        Common Location      818                    818          818   
         Geo-Override         469                    469          469   
         Intersection        3701                   3701         3701   
         Intersectioon          1                      1            1   
         Premise Address     5059                   5059         5059   
CA       1                      3                      3            3   

                          CallTime  CallDateTime  Disposition  Address  City  \
AgencyId AddressType                                                           
1        Common Location       818           818          818      818   817   
         Geo-Override          469           469          469      469   151   
         Intersection         3701          3701         3701     3701  3701   
         Intersectioon           1             1            1        1     1   
         Premise Address      5059          5059         5059     5059  5059   
CA       1                       3             3            3        3     1   

                          State  
AgencyId AddressType             
1        Common Location    818  
         Geo-Override       469  
         Intersection      3701  
         Intersectioon        1  
         Premise Address   5059  
CA       1                    0

In [13]:
""" observar muestras de las caracteristicas que poseen valores desconocidos"""
df[(df['AgencyId']=='CA') & (df['AddressType']=='1') ]

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallTime,CallDateTime,Disposition,Address,City,State,AgencyId,AddressType
5771,160942112,Auto Boost / Strip,2016-04-03T00:00:00,14:30,2016-04-03T14:30:00,REP,Martin Luther King Dr/bowling Green Dr,NaN,NaN,CA,1
8021,160952280,Auto Boost / Strip,2016-04-04T00:00:00,14:46,2016-04-04T14:46:00,REP,Martin Luther King Dr/nancy Pelosi Dr,S,NaN,CA,1
8473,160953118,Auto Boost / Strip,2016-04-04T00:00:00,18:11,2016-04-04T18:11:00,REP,Conservatory Drive E/john F Kennedy Dr,NaN,NaN,CA,1


In [14]:
"""eliminar registros obtenidos con datos faltantes y valores no conocidos """
df.drop([5771, 8021, 8473], axis=0, inplace=True)

En este apartado se procede a eliminar ciertos valores de tipo NaN, así como también, valores que se desconocen o que no son muy comunes en determinadas características. Para el primer caso, se elimina la columna Range debido a que sus datos presentan valores Nan. Es decir, no existe otro valor adicional para hacer una referencia en el caso de aplicar un metodo de sustición. Por otro lado, se eliminan las muestras que poseen valores desconocidos o que se pueden considerar como atípicos puesto que son distintos a la mayoria de las muestras. Para este caso, las columnas 'AgencyId' y 'AddressType', poseen valores CA y 1 respectivamente, donde las muestras restantes poseen otros valores del mismo tipo en su mayoria. Por lo tanto, los filas que coinciden con los valores que se mencionaron en la ultima parte se eliminan, representando aproximadamente el 0.3%(0.29%) del total de las muestras. Es decir, ya no se cuentan con el 100% del total de las muestras si no con el 99.7% de las muestras que representaria a la cantidad de 1048 muestras totales,anteriormente de 1051.

In [15]:
""" verificar nueva dimension del dataset """
df.shape

(10048, 11)

In [16]:
""" comprobar datos faltantes en el resto del conjunto de datos"""
df.isnull().sum()

CrimeId                    0
OriginalCrimeTypeName      0
OffenseDate                0
CallTime                   0
CallDateTime               0
Disposition                0
Address                    0
City                     319
State                      0
AgencyId                   0
AddressType                0
dtype: int64

### 1.4 Relleno de datos o imputacion de datos faltantes

In [17]:
""" verificar nuevamentes los valores unicos de la columna City despues de la eliminacion de datos desconocidos"""
df['City'].unique()

array(['San Francisco', nan, 'Daly City', 'Treasure Isla', 'Yerba Buena',
       'SAN FRANCISCO', 'Presidio', 'Brisbane'], dtype=object)

#### 1.4.1 Estandarizar etiquetas

* **Columna City**

In [18]:
""" mostrar fila que contiene la etiqueta SAN FRANCISO en la columna City"""
df[(df['City']=='SAN FRANCISCO')]

,CrimeId,OriginalCrimeTypeName,OffenseDate,CallTime,CallDateTime,Disposition,Address,City,State,AgencyId,AddressType
3420,160931948,Pups,2016-04-02T00:00:00,14:04,2016-04-02T14:04:00,HAN,Van Ness Av/north Point St,SAN FRANCISCO,CA,1,Intersection


In [19]:
""" Cambiar a mayuscula la letra inicial de cada palabra en todas las etiquetas de la columna City"""
df['City'] = df['City'].apply(lambda x: x if pd.isna(x) else string.capwords(x) )

In [22]:
""" verificar nuevamentes los valores unicos de la columna City """
df['City'].unique()

array(['San Francisco', nan, 'Daly City', 'Treasure Isla', 'Yerba Buena',
       'Presidio', 'Brisbane'], dtype=object)

* **Columna Disposition**

In [23]:
""" mostrar etiquetas para estandarizar (Intersectioon->Intersection)"""
valores_unicos_etiquetas(['AddressType'])

,AddressType,Cantidad
0,Common Location,818
1,Geo-Override,469
2,Intersection,3701
3,Intersectioon,1
4,Premise Address,5059


In [24]:
""" reemplazar etiqueta Intersectioon por Intersection """
df['AddressType']= df['AddressType'].apply(lambda x: x if not 'Intersectioon' else x.replace('Intersectioon','Intersection') )

In [25]:
""" comprobar etiquetas cambiadas(Intersectioon->Intersection)"""
valores_unicos_etiquetas(['AddressType'])

,AddressType,Cantidad
0,Common Location,818
1,Geo-Override,469
2,Intersection,3702
3,Premise Address,5059


#### 1.4.2 Imputacion de datos con la moda

* **Columna City**

In [26]:
""" crear instancia de imputacion para reemplazar datos Nan por la moda """
impu_moda = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
""" definir columna a imputar """
impu_moda= impu_moda.fit(df[['City']])
""" transformar datos aprendidos para la imputacion de datos faltantes """
df['City'] = impu_moda.transform(df[['City']])

In [27]:
""" verificar etiquetas despues de aplicar la imputacion de datos en la columna City """
valores_unicos_etiquetas(['City'])

,City,Cantidad
0,Brisbane,1
1,Daly City,5
2,Presidio,3
3,San Francisco,9985
4,Treasure Isla,51
5,Yerba Buena,3


* **Columna Disposition**

In [27]:
""" verificar etiquetas para aplicar imputacion de datos en la etiqueta '22' puesto que es un valor atipico """
valores_unicos_etiquetas(['Disposition'])

,Disposition,Cantidad
0,22,77
1,ABA,97
2,ADM,48
3,ADV,1142
4,ARR,65
5,CAN,353
6,CIT,1425
7,CRT,2
8,GOA,1273
9,HAN,2820


In [30]:
""" cambiar el valor de la etiqueta 22 a dato NaN para hacer la imputacion por KNN """
df.loc[df.Disposition=='22','Disposition']= np.nan

In [32]:
""" comprobar la existencia de los datos faltantes que fueron reemplazados por la etiqueta 22"""
df['Disposition'].isnull().sum()

77

In [34]:
""" definir columna para aprender datos e imputar por moda """
impu_moda_disposition= impu_moda.fit(df[['Disposition']])
""" transformar datos aprendidos para la imputacion de datos faltantes """
df['Disposition'] = impu_moda_disposition.transform(df[['Disposition']])

In [37]:
""" comprobar nuevamente si hay existencia de datos faltantes luego de la imputacion por moda"""
df['Disposition'].isnull().sum()

0

In [36]:
""" comprobar datos faltantes en el resto del conjunto de datos"""
df.isnull().sum()

CrimeId                  0
OriginalCrimeTypeName    0
OffenseDate              0
CallTime                 0
CallDateTime             0
Disposition              0
Address                  0
City                     0
State                    0
AgencyId                 0
AddressType              0
dtype: int64